In [32]:
import joblib
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text


# Load the saved model
rf_model = joblib.load("random_forest_model.pkl")
print("Model loaded successfully!")

Model loaded successfully!


I imported the saved model, and lets import calidation dataset as well.

In [20]:
val_data = pd.read_csv("validation_set.csv")

# Separate features and target variable
X_val = val_data.drop(columns=['target'])  # Assuming 'target' is the name of your dependent variable
y_val = val_data['target']

In [22]:
y_val_pred = rf_model.predict(X_val)

# Print validation accuracy
from sklearn.metrics import accuracy_score  

val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation Accuracy: 0.8889


c:\Users\Sanjeeb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ALthough, the testing accuracy was around 77.76% but the validation accuracy is 88.89%. 

Now, Lets saved the predicted data in database in analytics schema.

In [27]:
val_data['predicted_target'] = y_val_pred

In [25]:
host = r'127.0.0.1' 
db = r'MSDS610' 
user = r'postgres' 
pw = r'1234' 
port = r'5433' 
schema = r'analytics'

In [26]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [29]:
with db_conn.connect() as connection:
    val_data.to_sql('validation_predictions', con=connection, schema='analytics', if_exists='replace', index=False)

In [31]:
schema = "analytics"  
table_name = "validation_predictions"  

sql = f'SELECT * FROM "{schema}"."{table_name}"'

data = pd.read_sql(sql, db_conn, index_col=None)
print(data.head())  

   heart_rate_reserve       age   thalach  stress_impact  bp_chol_ratio   
0           -0.448281 -2.050205  1.111327       0.363597       0.196626  \
1           -0.766999  0.275576  0.649594       0.367810      -0.694744   
2            0.428194  0.624443 -0.620170      -0.394743       0.865153   
3           -2.002032  1.671044  1.380671       0.245633       0.419468   
4            0.268835  0.508154 -0.427782       0.001662      -0.249059   

   trestbps   oldpeak  slope_oldpeak_product  target  predicted_target  
0  0.196626 -0.686558              -0.624070       2                 2  
1 -0.694744 -0.185423              -0.367864       1                 1  
2  0.865153 -0.686558              -0.624070       1                 3  
3  0.419468  1.317981               1.425580       3                 3  
4 -0.249059 -0.586331              -0.572829       3                 3  


# Summary
After experimenting with different parameters, hyperparameter tuning, and trying multiple models, including XGBClassifier, the highest test accuracy achieved was 77.76%. Based on the results, I finalized a model using 8 important features and split the dataset into 70% training, 15% testing, and 15% validation. The trained model and validation set were saved, and in a new notebook, the model was tested on the validation set, achieving a validation accuracy of 88.89%. After that the predicted data was stored in analytics schema of database.

# Analysis
Yes, the severity of heart disease can be predicted using patient attributes with models like Random Forest and XGBoost, leveraging key features such as age, blood pressure, cholesterol, and heart rate. Among physiological factors, age contributes the most to heart disease risk, followed by blood pressure, as aging leads to arterial stiffness and increased cardiovascular strain, while high blood pressure adds stress to the heart and vessels. Other important factors include cholesterol levels, stress impact, and heart rate reserve, all playing a role in assessing heart disease severity.